In [2]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np
import pandas as pd
import pickle

In [3]:
with open('../data/dataset.pkl', 'rb') as f:
    features, labels = pickle.load(f)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [4]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

In [26]:
class TCN_block(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, dilation, padding, stride=1, dropout=0.2):
        super(TCN_block, self).__init__()
        self.conv = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
# [n_outputs, n_inputs, kernel_size]
        self.chomp = Chomp1d(padding)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv, self.chomp, self.relu, self.dropout)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.init_weights()

    def init_weights(self):
        self.conv.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

In [70]:
class TCN_Model(nn.Module):
    def __init__(self, batch_size, firs_layer_outputs, kernel_size, dilation):
        super(TCN_Model, self).__init__()
        padding = dilation * (kernel_size - 1)
        self.tcn1 = TCN_block(1, firs_layer_outputs, kernel_size, dilation, padding)
        self.tcn2 = TCN_block(firs_layer_outputs, firs_layer_outputs * 5, kernel_size, dilation, padding)
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(n_outputs//5, n_outputs//15)
        self.dense2 = nn.Linear(n_outputs//15, n_outputs//30)
        self.dense3 = nn.Linear(n_outputs//30, n_outputs//60)
        self.dense4 = nn.Linear(n_outputs//60, n_outputs//120)
        self.dense_layers = nn.Sequential(self.dense1, nn.ReLU(),
                                          self.dense2, nn.ReLU(),
                                          self.dense3, nn.ReLU(),
                                          self.dense4, nn.Sigmoid())
        self.init_weights()
        
    def init_weights(self):
        self.dense1.weight.data.normal_(0, 0.01)
        self.dense2.weight.data.normal_(0, 0.01)
        self.dense3.weight.data.normal_(0, 0.01)
        self.dense4.weight.data.normal_(0, 0.01)
    
    def forward(self, x):
        conv_out = self.conv_layers(x)
        torch.reshape(conv_out,(1,len(conv_out)))
        return self.dense_layers(conv_out)

In [60]:
def outlier_handler(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    IQR = q3-q1
    lwr_bound = q1-(1.5*IQR)
    upr_bound = q3+(1.5*IQR)
    df = np.where(df > upr_bound, df.median(), np.where(df < lwr_bound, df.median(), df))
    return df

numeric_transformer = Pipeline([
    ('Outlier_handler', FunctionTransformer(outlier_handler)),
    ('Imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline([
    ('Imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('Binary_encoder', OneHotEncoder(sparse=False, drop='if_binary', handle_unknown='ignore'))
    # ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

numeric_features = X_train.select_dtypes(['int64', 'float64']).columns
cat_cols = X_train.select_dtypes('category').columns
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, cat_cols)
])

proc = Pipeline([
    ("preprocessor", preprocessor),
    ("scaler", StandardScaler())
])

In [74]:
processed_data = proc.fit_transform(X_train, y_train)
data_shape = processed_data.shape
processed_data = processed_data.reshape([data_shape[0], 1, data_shape[1]])
batch_size = 40

training_set = []
for i in range(len(processed_data)):
   training_set.append([processed_data[i], y_train.to_numpy()[i]])

training_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=False, num_workers=4)

In [71]:
firs_layer_outputs = 5
kernel_size = 2
dilation = 1

model = TCN_Model(firs_layer_outputs, kernel_size, dilation)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [72]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        
        outputs = model(inputs.float())

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [73]:
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

model = model.float()
model.train(True)
avg_loss = train_one_epoch(epoch_number, writer)

TypeError: reshape(): argument 'shape' must be tuple of ints, but found element of type builtin_function_or_method at pos 2